## Step 1: Perform RFM Analysis
- Calculate Recency, Frequency, and Monetary values for each customer

In [17]:
import pandas as pd

import os
print(os.listdir())

df = pd.read_csv('flo_data_20k.csv')
print(df.columns)

['.DS_Store', 'RFM_Clustering_Assignment.ipynb', 'Untitled.ipynb', 'Week 3', 'Week 4', 'flo_data_20k.csv', 'Week 5', '.ipynb_checkpoints', 'Week 6']
Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')


In [19]:
import pandas as pd

df = pd.read_csv("flo_data_20k.csv")
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [37]:
df["last_order_date"] = pd.to_datetime(df["last_order_date"])
ref_date = df["last_order_date"].max() + pd.Timedelta(days=1)

In [41]:
df["Recency"] = (ref_date - df["last_order_date"]).dt.days
df["Frequency"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["Monetary"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

In [43]:
df[["master_id", "Recency", "Frequency", "Monetary"]].head()

,master_id,Recency,Frequency,Monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,94,5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,104,21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,185,5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,134,2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,85,2.0,209.98


In [49]:
rfm = df[["master_id", "Recency", "Frequency", "Monetary"]].copy()
rfm.set_index("master_id", inplace=True)

In [51]:
rfm["R"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1]).astype(int)
rfm["F"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm["M"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5]).astype(int)
rfm["RFM_Score"] = rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)

In [53]:
rfm["RFM_Score"].value_counts().head(10)
rfm["Segment"] = "Others"

rfm.loc[rfm["RFM_Score"] == "555", "Segment"] = "Champions"
rfm.loc[rfm["R"] >= 4, "Segment"] = "Recent"
rfm.loc[rfm["F"] >= 4, "Segment"] = "Frequent"
rfm.loc[rfm["M"] >= 4, "Segment"] = "High Value"
rfm.loc[(rfm["R"] <= 2) & (rfm["F"] <= 2), "Segment"] = "At Risk"

In [60]:
rfm[["RFM_Score", "Segment"]].head(10)

,RFM_Score,Segment
master_id,,
cc294636-19f0-11eb-8d74-000d3a38a36f,344,High Value
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,355,High Value
69b69676-1a40-11ea-941b-000d3a38a36f,243,Frequent
1854e56c-491f-11eb-806e-000d3a38a36f,311,Others
d6ea1074-f1f5-11e9-9346-000d3a38a36f,311,Others
e585280e-aae1-11e9-a2fc-000d3a38a36f,421,Recent
c445e4ee-6242-11ea-9d1a-000d3a38a36f,232,Others
3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,111,At Risk
cfbda69e-5b4f-11ea-aca7-000d3a38a36f,345,High Value


- Assign RFM scores using quintile-based scoring (1–5)

In [63]:
rfm = df[["master_id", "Recency", "Frequency", "Monetary"]].copy()
rfm.set_index("master_id", inplace=True)

In [65]:
rfm["R"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1]).astype(int)

In [67]:
rfm["F"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]).astype(int)

In [69]:
rfm["M"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5]).astype(int)

In [71]:
rfm["RFM_Score"] = (
    rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)
)

In [73]:
rfm[["Recency", "R", "Frequency", "F", "Monetary", "M", "RFM_Score"]].head()

,Recency,R,Frequency,F,Monetary,M,RFM_Score
master_id,,,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,94,3,5.0,4,939.37,4,344
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,104,3,21.0,5,2013.55,5,355
69b69676-1a40-11ea-941b-000d3a38a36f,185,2,5.0,4,585.32,3,243
1854e56c-491f-11eb-806e-000d3a38a36f,134,3,2.0,1,121.97,1,311
d6ea1074-f1f5-11e9-9346-000d3a38a36f,85,3,2.0,1,209.98,1,311


- Segment customers into High, Mid, and Low Value groups based on total RFM score

In [76]:
rfm["RFM_Total"] = rfm[["R", "F", "M"]].sum(axis=1)

In [78]:
def value_segment(score):
    if score >= 12:
        return "High Value"
    elif score >= 8:
        return "Mid Value"
    else:
        return "Low Value"

rfm["Value_Segment"] = rfm["RFM_Total"].apply(value_segment)

In [80]:
rfm[["R", "F", "M", "RFM_Total", "Value_Segment"]].head()

,R,F,M,RFM_Total,Value_Segment
master_id,,,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,3,4,4,11,Mid Value
f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,3,5,5,13,High Value
69b69676-1a40-11ea-941b-000d3a38a36f,2,4,3,9,Mid Value
1854e56c-491f-11eb-806e-000d3a38a36f,3,1,1,5,Low Value
d6ea1074-f1f5-11e9-9346-000d3a38a36f,3,1,1,5,Low Value


In [82]:
rfm["Value_Segment"].value_counts()

Value_Segment
Mid Value     7883
Low Value     7103
High Value    4959
Name: count, dtype: int64

## Step 1: RFM Analysis

### 1. Calculate Recency, Frequency, and Monetary Values
For each customer, we calculated the following metrics:
- **Recency**: Number of days since the customer's last purchase (compared to a reference date).
- **Frequency**: Total number of transactions made by the customer (both online and offline).
- **Monetary**: Total amount of money spent by the customer (online and offline combined).

### 2. Assign RFM Scores Using Quintile-Based Scoring (1–5)
Each of the RFM metrics was scored based on quintiles:
- **Recency**: Lower recency (more recent purchase) received a higher score (5 = most recent).
- **Frequency & Monetary**: Higher values received higher scores (5 = most frequent or highest spending).
- These scores were then concatenated into a 3-digit string called `RFM_Score` (e.g., "355", "421").

### 3. Segment Customers Based on Total RFM Score
We summed the individual RFM scores to get a total RFM score per customer.
Based on this total score, customers were segmented as follows:
- **High Value**: RFM Total between 12–15
- **Mid Value**: RFM Total between 8–11
- **Low Value**: RFM Total between 3–7

Final customer distribution by segment:
- High Value: 4,959 customers  
- Mid Value: 7,883 customers  
- Low Value: 7,103 customers